In [314]:
import os
import re
import math
import gzip
import shutil
import warnings
from pathlib import Path
from datetime import datetime, timedelta, date
from ftplib import FTP
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import rioxarray
from pyproj import CRS
from pyresample.geometry import AreaDefinition
try:
    from satpy.readers.core._geos_area import get_area_extent
except ImportError:
    from satpy.readers._geos_area import get_area_extent
from enhanced_steps import EnhancedStepsNowcast
warnings.filterwarnings("ignore")
import os
import re
import math
import gzip
import shutil
import warnings
from pathlib import Path
from datetime import datetime, timedelta, date
from ftplib import FTP
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import rioxarray
from pyproj import CRS
from pyresample.geometry import AreaDefinition
try:
    from satpy.readers.core._geos_area import get_area_extent
except ImportError:
    from satpy.readers._geos_area import get_area_extent
from enhanced_steps import EnhancedStepsNowcast
from PIL import Image, ImageSequence, ImageDraw, ImageFont
warnings.filterwarnings("ignore")
print("Toutes les bibliothèques ont été importées avec succès!")

Toutes les bibliothèques ont été importées avec succès!


In [315]:
username = "xxxxxxx"                      # Remplacez par votre nom d'utilisateur réel
password = "xxxxxxx"                # Remplacez par votre mot de passe réel

# Mode de sélection : "Selectionner_données_cibles" ou "Actuel"
selection_mode = "Selectionner_données_cibles"  
# selection_mode = "Actuel"  

target_datetime = datetime(2025, 7, 26, 15, 30)   # Point de départ pour la sélection des données
n_steps = 10                                      # Nombre d'étapes à traiter
region = {"lat_min": 4, "lat_max": 16, "lon_min": -6, "lon_max": 3}  # Région d'intérêt (latitude/longitude)
ensemble_number = 1                              # Numéro du membre d'ensemble à utiliser

points_dinteret = [
    {"nom": "Nouna", "lat": 12.76, "lon": -3.84},        # Point d'intérêt 1
    {"nom": "Kante", "lat": 9.961861, "lon": 1.042944},  # Point d'intérêt 2
]

make_gif = True                                 # True pour créer une animation GIF, False sinon
make_timeseries_Excel = True                    # True pour générer un fichier Excel avec les séries temporelles
make_graph = True                               # True pour créer des graphiques

# Paramètres de visualisation
bounds = [0, 0.5, 2, 5, 10, 15, 25, 40, 100]                      # Bornes pour la colorbar
colors = ["#ffffff", "#add8e6", "#0000ff", "#00ff00",
          "#ffff00", "#ffa500", "#ff0000", "#ff69b4", "#ff69b4"]  # Couleurs pour chaque intervalle

In [316]:
# Créer des dossiers pour nos données
data_folder = Path("./h60b_data")
raw_folder = data_folder / "raw"
processed_folder = data_folder / "processed"
nowcasting_folder = data_folder / "nowcast"

# Créer les répertoires s'ils n'existent pas encore
raw_folder.mkdir(parents=True, exist_ok=True)
processed_folder.mkdir(parents=True, exist_ok=True)
Path(nowcasting_folder).mkdir(parents=True, exist_ok=True)

In [317]:
# Détails du serveur FTP HSAF
ftp_server = "ftphsaf.meteoam.it"
ftp_directory = "./h60B/h60_cur_mon_data/"

def parse_h60b_timestamp(filename):
    try:
        parts = filename.split("_")
        date_str = parts[1]                  # YYYYMMDD
        time_str = parts[2].split(".")[0]    # HHMM
        timestamp = datetime.strptime(f"{date_str}{time_str}", "%Y%m%d%H%M")
        return timestamp
    except:
        return None
    
def list_h60b_files_full():
    try:
        print("Connexion au serveur FTP HSAF...")
        with FTP(ftp_server) as ftp:
            # Se connecter avec les identifiants
            ftp.login(username, password)
            print("Connexion réussie!")
            ftp.cwd(ftp_directory)
            files = []
            ftp.dir(lambda line: files.append(line.split()[-1]))
            h60b_files = [f for f in files if f.startswith("h60")]
            timestamps_files = [(parse_h60b_timestamp(f), f) 
                                for f in h60b_files 
                                if parse_h60b_timestamp(f) is not None]
            
            if not timestamps_files:
                print("Aucun fichier avec timestamp valide trouvé.")
                return []
            
            timestamps_files.sort()
            
            first_file = timestamps_files[0]
            last_file  = timestamps_files[-1]
            print(f"Dernier fichier disponible : {last_file[1]} ({last_file[0]})")
            return [f[1] for f in timestamps_files]
    
    except Exception as e:
        print(f"Erreur lors de la connexion FTP : {e}")
        print("Assurez-vous de vous être inscrit sur https://hsaf.meteoam.it/ et d'avoir des identifiants valides")
        return []
    
all_files = list_h60b_files_full()
timestamps_files = [(parse_h60b_timestamp(f), f) for f in all_files if parse_h60b_timestamp(f)]
timestamps_files.sort()

Connexion au serveur FTP HSAF...
Connexion réussie!
Dernier fichier disponible : h60_20250905_0800_fdk.nc.gz (2025-09-05 08:00:00)


In [318]:
def select_files_before_date(target_time=None, n_steps=10, timestamps_files=timestamps_files, actuel=False):
    if not timestamps_files:
        print("Aucun fichier disponible pour la sélection.")
        return []

    if actuel:
        selectionnes = timestamps_files[-n_steps:] if n_steps <= len(timestamps_files) else timestamps_files
        return [f[1] for f in selectionnes]

    if target_time is None:
        print("Erreur : target_time doit être fourni si 'actuel' est False.")
        return []

    fichiers_selectionnes = []
    ts_courant = target_time

    for _ in range(n_steps):
        fichier_precedent = None
        for ts, f in reversed(timestamps_files):
            if ts <= ts_courant:
                fichier_precedent = (ts, f)
                break

        if fichier_precedent:
            fichiers_selectionnes.append(fichier_precedent[1])
            ts_courant = fichier_precedent[0] - timedelta(minutes=15)
        else:
            print(f"Aucun fichier disponible avant {ts_courant}")
            break

    return fichiers_selectionnes[::-1]

if selection_mode == "Selectionner_données_cibles":
    selected_files = select_files_before_date(target_datetime, n_steps=n_steps, actuel=False)
elif selection_mode == "Actuel":
    selected_files = select_files_before_date(target_datetime, n_steps=n_steps, actuel=True)


In [319]:
def download_h60b_file(filename, output_folder=None):

    target_folder = Path(output_folder) if output_folder else raw_folder
    target_folder.mkdir(parents=True, exist_ok=True)

    gz_path = target_folder / filename
    nc_path = target_folder / filename.replace(".gz", "")

    if nc_path.exists():
        print(f"Le fichier existe déjà : {nc_path.name}")
        return str(nc_path)

    try:
        print(f"Téléchargement de {filename}...")

        with FTP(ftp_server) as ftp:
            ftp.login(username, password)
            ftp.cwd(ftp_directory)

            with open(gz_path, "wb") as f:
                ftp.retrbinary(f"RETR {filename}", f.write)

        file_size = gz_path.stat().st_size / (1024*1024)  # Taille en MB
        with gzip.open(gz_path, "rb") as f_in:
            with open(nc_path, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)
        gz_path.unlink()

        final_size = nc_path.stat().st_size / (1024*1024)  # Taille finale en MB

        return str(nc_path)

    except Exception as e:
        print(f"Erreur lors du téléchargement de {filename} : {e}")
        return None

In [320]:
def preprocess_file(raw_file: str, output_folder=None):
    
    target_folder = Path(output_folder) if output_folder else processed_folder
    target_folder.mkdir(parents=True, exist_ok=True)
    filename = Path(raw_file).name

    # Extraire le timestamp depuis le nom du fichier et définir le chemin de sortie
    timestamp = parse_h60b_timestamp(filename)
    hsaf_filename = f"HSAF-H60B_{timestamp.strftime('%Y%m%dT%H%M%S')}.nc"
    output_path_HSAF = target_folder / hsaf_filename

    # Vérifier si le fichier prétraité existe déjà
    if output_path_HSAF.exists():
        print(f"Fichier déjà prétraité : {hsaf_filename} -> Ignorer le prétraitement")
        return str(output_path_HSAF)

    # --- Définitions des systèmes de coordonnées
    crs_in = "+proj=geos +a=6378.169 +b=6356.584 +h=35785.831 +lat_0=0 +lon_0=0.000000"  # CRS d’origine (satellite)
    crs_out = "EPSG:4326"  # CRS de sortie : latitude/longitude

    ds = xr.open_dataset(raw_file, decode_coords="all", decode_cf=False)
    ds = ds.rename({"nx": "x", "ny": "y"})
    rr_raw = ds["rr"]
    scale_factor = rr_raw.encoding.get("scale_factor", 0.1)
    add_offset  = rr_raw.encoding.get("add_offset", 0.0)
    ds["rr"] = rr_raw.astype("float32") * scale_factor + add_offset
    ds["rr"].encoding.clear()
    ds["rr"].attrs.clear()
    ds = ds[["rr"]].astype("float32")

    source_crs = CRS.from_proj4(ds.attrs["gdal_projection"])
    crs_in = source_crs.to_proj4()  # CRS exact du fichier
    crs_out = "EPSG:4326"
    cgms_projection = (
        "+proj=geos +coff=1856.000000 +cfac=13642337.000000 "
        "+loff=1856.000000 +lfac=13642337.000000 "
        "+spp=0.000000 +r_eq=6378.169000 +r_pol=6356.583800 +h=42164.000000"
    )
    matches = re.findall(r"\+?(\w+)\s*=\s*([^\s]+)", cgms_projection)
    parse_dict = {k: v for k, v in matches}
    cd = source_crs.to_dict()
    area_extent = get_area_extent({
        "scandir": "N2S",                                                           # scandir : direction de balayage du satellite (N2S = Nord vers Sud)
        "h": float(parse_dict["h"]) * 1000 - float(parse_dict["r_eq"]) * 1000,      # h : hauteur du satellite au-dessus de la Terre       
        "loff": float(parse_dict["loff"]),                                          # loff, coff : décalages de ligne et de colonne (début du raster)
        "coff": float(parse_dict["coff"]),                                          
        "lfac": float(parse_dict["lfac"]),                                          # lfac, cfac : facteurs d'échelle pour les lignes et colonnes
        "cfac": float(parse_dict["cfac"]),
        "ncols": ds.x.size,                                                         # ncols, nlines : nombre de colonnes et lignes du raster
        "nlines": ds.y.size,
    })

    area_def_src = AreaDefinition(
        "areaD",
        cd["proj"],
        "areaD",
        {"lon_0": cd["lon_0"], "a": cd["a"], "b": cd["b"], "h": cd["h"], "proj": cd["proj"]},
        ds.y.size,
        ds.x.size,
        (area_extent[0], area_extent[1], area_extent[2], area_extent[3]),
    )

    # Créer de nouvelles coordonnées X et Y
    x, y = area_def_src.get_proj_coords()
    new_x_coords = np.linspace(x.min(), x.max(), num=ds.sizes["x"])
    new_y_coords = np.linspace(y.max(), y.min(), num=ds.sizes["y"])
    ds = ds.assign_coords(y=("y", new_y_coords), x=("x", new_x_coords))

    # Écrire le CRS qui correspond aux unités des coordonnées
    ds = ds.rio.write_crs(crs_in)
    ds = ds.rename({"rr": "precip_intensity"}).sortby("y")
    ds = ds.expand_dims("time").assign_coords(time=("time", [timestamp]))
    ds = ds.transpose("time", "y", "x")
    ds.precip_intensity.attrs = {
        "standard_name": "precipitation_flux",
        "long_name": "Flux de précipitation dérivé des propriétés optiques des nuages",
        "units": "kg m-2 h-1",
    }

    ds["precip_intensity"] = ds["precip_intensity"].where(ds["precip_intensity"] >= 0, np.nan)
    da = ds["precip_intensity"].rio.write_nodata(np.nan, encoded=True)
    ds["precip_intensity"] = da
    ds = ds.rio.reproject(crs_out, nodata=np.nan)

    # Nettoyer l'encodage après reprojection
    for var_name in ds.data_vars:
        ds[var_name].encoding.clear()
        for attr in ["_FillValue", "missing_value", "fill_value", "FillValue"]:
            ds[var_name].attrs.pop(attr, None)

    # Ordre final des dimensions
    ds = ds.transpose("time", "y", "x")

    # Définir les attributs CF pour les coordonnées
    ds.x.attrs = {"standard_name": "longitude", "long_name": "longitude", "units": "degrees_east", "axis": "X"}
    ds.y.attrs = {"standard_name": "latitude", "long_name": "latitude", "units": "degrees_north", "axis": "Y"}
    ds.time.attrs = {"standard_name": "time", "long_name": "time", "axis": "T"}

    # Définir les attributs globaux du dataset
    ds.attrs = {
        "Conventions": "CF-1.6",
        "title": "RAINSAT H60B MSG SEVIRI Precipitation",
        "source": "EUMETSAT H-SAF H60B",
        "creator": "HKV services",
        "creation_date": date.today().strftime("%Y-%m-%d"),
        "time_coverage_start": timestamp.strftime("%Y-%m-%dT%H:%M:%S"),
        "time_coverage_end": (timestamp + timedelta(minutes=15)).strftime("%Y-%m-%dT%H:%M:%S"),
        "geospatial_lat_min": float(ds.y.min()),
        "geospatial_lat_max": float(ds.y.max()),
        "geospatial_lon_min": float(ds.x.min()),
        "geospatial_lon_max": float(ds.x.max()),
        "crs": crs_out,
        "product_details": "https://hsaf.meteoam.it/Products/Detail?prod=H60B",
        "data_source": "hsaf-h60b",
    }

    # Encodage pour NetCDF
    encoding = {
        "precip_intensity": {"dtype": "float32", "zlib": True, "complevel": 4, "_FillValue": -999.0},
        "time": {"units": "minutes since 1970-01-01 00:00:00", "dtype": "float64"},
    }

    # Sauvegarder le fichier NetCDF prétraité
    ds.to_netcdf(output_path_HSAF, encoding=encoding)
    ds.close()
    print(f"Fichier H60B prétraité avec succès : {hsaf_filename}")
    return str(output_path_HSAF)

In [321]:
# Boucle finale pour télécharger et prétraiter les fichiers sélectionnés
for selected_file in selected_files:
    downloaded_file = download_h60b_file(selected_file)
    timestamp = parse_h60b_timestamp(Path(downloaded_file).name)
    hsaf_filename = f"HSAF-H60B_{timestamp.strftime('%Y%m%dT%H%M%S')}.nc"
    output_path_HSAF = processed_folder / hsaf_filename

    if output_path_HSAF.exists():
        print(f"{hsaf_filename} existe déjà, passage au fichier suivant\n" + "-"*40)
        continue
    
    preprocess_file(downloaded_file)
    print(f"{selected_file} traité\n" + "-"*40)

Le fichier existe déjà : h60_20250726_1315_fdk.nc
HSAF-H60B_20250726T131500.nc existe déjà, passage au fichier suivant
----------------------------------------
Le fichier existe déjà : h60_20250726_1330_fdk.nc
HSAF-H60B_20250726T133000.nc existe déjà, passage au fichier suivant
----------------------------------------
Le fichier existe déjà : h60_20250726_1345_fdk.nc
HSAF-H60B_20250726T134500.nc existe déjà, passage au fichier suivant
----------------------------------------
Le fichier existe déjà : h60_20250726_1400_fdk.nc
HSAF-H60B_20250726T140000.nc existe déjà, passage au fichier suivant
----------------------------------------
Le fichier existe déjà : h60_20250726_1415_fdk.nc
HSAF-H60B_20250726T141500.nc existe déjà, passage au fichier suivant
----------------------------------------
Le fichier existe déjà : h60_20250726_1430_fdk.nc
HSAF-H60B_20250726T143000.nc existe déjà, passage au fichier suivant
----------------------------------------
Le fichier existe déjà : h60_20250726_14

In [322]:
settings = {
    "data_source": "h60b",     # Source des données
    "ensemble": 5,             # Nombre de membres de l'ensemble
    "n_input_files": 10,       # Nombre de fichiers d'entrée utilisés
    "n_lead_times": 12,        # Nombre de pas de temps à prévoir, chaque pas de temps dure 15 minutes
    "frequency": 15,           # Minutes entre chaque pas de temps
    "transform": "dB",         # Transformation utilisée par pysteps ("dB", "log", ...)
    "threshold": 0.05,         # Seuil (mm/h) pour définir pluie / pas de pluie
    "buffer_distance": 5000,    # Distance en mètres autour du pays pour le découpage
    "crs_out": "EPSG:4326",    # Système de coordonnées de sortie
    "norain_thr": 0.005,       # Paramètres pour la méthode Norain
    "zerovalue": -15.0,        # Paramètres pour la méthode Norain
    "max_workers": 2,          # Nombre de processeurs à utiliser
}

In [323]:
def match_files_by_timestamp(selected_files, processed_folder):
    processed_folder = Path(processed_folder)
    all_files = list(processed_folder.glob("*"))

    ts_list = []
    pattern_selected = r"(\d{8}_\d{4})"
    for f in selected_files:
        match = re.search(pattern_selected, f)
        if match:
            dt = datetime.strptime(match.group(), "%Y%m%d_%H%M")
            ts_list.append(dt)

    matched_files = []
    pattern_processed = r"(\d{8}T\d{6})"  
    for f in all_files:
        match = re.search(pattern_processed, f.name)
        if match:
            dt_f = datetime.strptime(match.group(), "%Y%m%dT%H%M%S")
            if dt_f in ts_list:
                matched_files.append(f)

    matched_files.sort(key=lambda x: re.search(pattern_processed, x.name).group())
    return matched_files

observations_input_nowcast = match_files_by_timestamp(selected_files, processed_folder)

def open_as_time_stack(input_files):
    if not input_files:
        raise FileNotFoundError("Aucun fichier d'entrée trouvé.")   
    datasets = xr.open_mfdataset(input_files)                       
    return datasets

input_dataset = open_as_time_stack(observations_input_nowcast)

In [324]:
nc_path = Path("h60b_data/nowcast/pysteps_h60b_latest.nc")
if nc_path.exists():
    nc_path.unlink()

steps_settings = {
    "datafolder": data_folder,                      # Dossier contenant les fichiers HSAF prétraités
    "ensemble": settings["ensemble"],               # Nombre de membres de l'ensemble
    "n_lead_times": settings["n_lead_times"],       # Nombre de pas de temps à prévoir
    "frequency": settings["frequency"],             # Fréquence entre chaque pas de temps (minutes)
    "transform": settings["transform"],             # Transformation PySTEPS ("dB", "log", etc.)
    "threshold": settings["threshold"],             # Seuil pour distinguer pluie / pas de pluie
    "buffer_distance": settings["buffer_distance"], # Tampon autour de la zone d'intérêt
    "crs_out": settings["crs_out"],                 # Système de coordonnées de sortie
    "norain_thr": settings["norain_thr"],           # Paramètre pour "no rain"
    "zerovalue": settings["zerovalue"],             # Valeur minimale pour "no rain"
    "max_workers": settings["max_workers"],         # Nombre de CPU à utiliser
}

engine = EnhancedStepsNowcast(steps_settings, "h60b")

dataset_roi = input_dataset.sel(
    x=slice(region["lon_min"], region["lon_max"]),
    y=slice(region["lat_max"], region["lat_min"])  # attention aux coordonnées inversées
)

nowcast_arrays, metadata = engine.nowcast_steps_pysteps(dataset_roi)

Rain fraction is: 0.12485211895308322, while minimum fraction is 0.005
unknown projection longlat
Inputs validated and initialized successfully.
Computing STEPS nowcast
-----------------------

Inputs
------
input dimensions: 287x215
km/pixel:         3.0
time step:        15 minutes

Methods
-------
extrapolation:          semilagrangian
bandpass filter:        gaussian
decomposition:          fft
noise generator:        nonparametric
noise adjustment:       no
velocity perturbator:   None
conditional statistics: no
precip. mask method:    incremental
probability matching:   cdf
FFT method:             numpy
domain:                 spatial

Parameters
----------
number of time steps:     12
ensemble size:            5
parallel threads:         2
number of cascade levels: 6
order of the AR(p) model: 2
precip. intensity threshold: -13.010299956639813
Nowcast components initialized successfully.
Rain fraction is: 0.11410744672230776, while minimum fraction is 0.0
Extrapolation complete a

In [325]:
output_path_nowcast = "/workspaces/Tools-for-weather-and-climate-services-in-Africa/2_Créer_un_Nowcast_avec_HSAF_H60B/h60b_data/nowcast"
Path(output_path_nowcast).mkdir(parents=True, exist_ok=True)
engine.settings["threddsdata"] = output_path_nowcast   
country_name = f"Togo_BurkinaFaso_{target_datetime.strftime('%Y%m%dT%H%M')}"

engine.export_nowcast_to_netcdf(
    country=country_name,
    nowcasting_arrays=nowcast_arrays,
    date_start=None,  
    metadata=metadata,
    reproject=True,
    data_source=settings["data_source"],
)

## Options de sauvegarde des résultats

In [326]:
# Créez des GIF pour les observations 
if make_gif:
    obs_files = []
    pattern = r"\d{8}T\d{4}"  

    for f in observations_input_nowcast:
        match = re.search(pattern, str(f))  
        if match:
            ts_str = match.group()
            matching = list(processed_folder.glob(f"*{ts_str}*.nc"))
            obs_files.extend(matching)
        else:
            print(f"Geen timestamp gevonden in: {f}")

    if not obs_files:
        print("Geen observatiebestanden gevonden voor de geselecteerde tijdstappen.")
    else:
        ds_obs = xr.open_mfdataset([str(f) for f in obs_files])

        # Region of Interest
        ds_roi = ds_obs.sel(
            x=slice(region["lon_min"], region["lon_max"]),
            y=slice(region["lat_max"], region["lat_min"])
        )

        var = ds_roi["precip_intensity"] if "precip_intensity" in ds_roi.data_vars else ds_roi["precipitation"]
        x = var["x"].values
        y = var["y"].values

        if y[0] > y[-1]:
            extent = [x.min(), x.max(), y.max(), y.min()]
            origin = "upper"
        else:
            extent = [x.min(), x.max(), y.min(), y.max()]
            origin = "lower"

        cmap = ListedColormap(colors)
        norm = BoundaryNorm(boundaries=bounds, ncolors=cmap.N, extend='max')
        cmap.set_bad("none")
        
        fig, ax = plt.subplots(figsize=(5, 6), subplot_kw={"projection": ccrs.PlateCarree()})
        plt.close(fig)
        ax.set_extent([x.min(), x.max(), y.min(), y.max()])
        ax.coastlines(resolution='10m', color='black')
        ax.add_feature(cfeature.BORDERS, edgecolor='black')

        last_frame = var.isel(time=-1)
        last_frame_flipped = np.ma.masked_invalid(last_frame.values[::-1, :]).astype(np.float32)

        img = ax.imshow(
            last_frame_flipped,
            extent=extent,
            origin=origin,
            cmap=cmap,
            norm=norm
        )

        cbar = fig.colorbar(img, ax=ax, fraction=0.05, pad=0.02, ticks=bounds)
        cbar.set_label("Précipitations (mm/h)")
        ax.set_title(np.datetime_as_string(last_frame.time.values, unit='m'))
        def update(i):
            frame = var.isel(time=i)
            frame_flipped = np.ma.masked_invalid(frame.values[::-1, :]).astype(np.float32)
            img.set_data(frame_flipped)
            ax.set_title(np.datetime_as_string(frame.time.values, unit='m'))
            return (img,)

        ani = FuncAnimation(fig, update, frames=var.sizes["time"], interval=600, blit=True)

        results_folder = Path("results")
        results_folder.mkdir(parents=True, exist_ok=True)

        last_time = var.time.values[9]
        last_time_dt = pd.to_datetime(last_time)
        safe_time_str = last_time_dt.strftime("%Y%m%dT%H%M")

        time_folder = results_folder / f"ens_{ensemble_number}_{safe_time_str}"
        time_folder.mkdir(parents=True, exist_ok=True)
        
        gif_filename = f"observations_animation_{safe_time_str}.gif"
        gif_path = time_folder / gif_filename
        ani.save(gif_path, writer="pillow", fps=2)

In [327]:
# Créez des GIF pour le nowcast 
if make_gif:
    ds_nowcast = xr.open_dataset(nowcasting_folder / "pysteps_h60b_latest.nc")
    varname = "precip_intensity" if "precip_intensity" in ds_nowcast.data_vars else "precipitation"
    var = ds_nowcast[varname]
    ds_nowcast.close()
    if "ens_number" in var.dims:
        var = var.isel(ens_number=ensemble_number)

    def fmt_time(tarr, i):
        v = tarr.isel(time=i).values
        try:
            return np.datetime_as_string(np.asarray(v, dtype="datetime64[m]"), unit='m')
        except Exception:
            try:
                return v.strftime("%Y-%m-%d %H:%M")
            except Exception:
                return str(v)

    cmap = ListedColormap(colors)
    norm = BoundaryNorm(boundaries=bounds, ncolors=cmap.N, extend='max')
    cmap.set_bad("none")
    xname = next((c for c in ("x", "lon", "longitude") if c in var.coords), None)
    yname = next((c for c in ("y", "lat", "latitude") if c in var.coords), None)
    x = var[xname].values
    y = var[yname].values

    if y[0] > y[-1]:
        extent = [x.min(), x.max(), y.max(), y.min()]
        origin = "upper"
    else:
        extent = [x.min(), x.max(), y.min(), y.max()]
        origin = "lower"

    fig, ax = plt.subplots(figsize=(5, 6), subplot_kw={"projection": ccrs.PlateCarree()})
    plt.close(fig) 
    ax.set_extent([x.min(), x.max(), y.min(), y.max()])
    ax.coastlines(resolution='10m', color='black')
    ax.add_feature(cfeature.BORDERS, edgecolor='black')

    first_frame = var.isel(time=0).where(var.isel(time=0) > 0)
    first_frame_flipped = np.ma.masked_invalid(first_frame.values[::-1, :]).astype(np.float32)

    img = ax.imshow(
        first_frame_flipped,
        extent=extent,
        origin=origin,
        cmap=cmap,
        norm=norm
    )

    cbar = fig.colorbar(img, ax=ax, fraction=0.05, pad=0.02, ticks=bounds)
    cbar.set_label("Précipitations (mm/h)")
    ax.set_title(fmt_time(var["time"], 0))

    def update(i):
        frame = var.isel(time=i).where(var.isel(time=i) > 0)
        frame_flipped = np.ma.masked_invalid(frame.values[::-1, :]).astype(np.float32)
        img.set_data(frame_flipped)
        ax.set_title(fmt_time(var["time"], i))
        return (img,)

    ani = FuncAnimation(fig, update, frames=var.sizes["time"], interval=600, blit=True)

    gif_filename = f"nowcast_animation_{target_datetime.strftime('%Y%m%dT%H%M')}.gif"
    gif_path = time_folder / gif_filename
    ani.save(gif_path, writer="pillow", fps=2)

In [328]:
## Créer Excel avec des observations de séries chronologiques
if make_timeseries_Excel:
    ds_obs = xr.open_mfdataset([str(f) for f in observations_input_nowcast])

    last_time = ds_obs.time.values[-1]
    last_time_str = np.datetime_as_string(last_time, unit='m')
    safe_time_str = last_time_str.replace(":", "").replace("-", "")

    data_dict = {}
    for pt in points_dinteret:
        varname = "precip_intensity" if "precip_intensity" in ds_obs.data_vars else "precipitation"
        rain_series = ds_obs.sel(
            x=ds_obs["x"].sel(x=pt["lon"], method="nearest"),
            y=ds_obs["y"].sel(y=pt["lat"], method="nearest")
        )[varname]
        data_dict[pt["nom"]] = rain_series.values

    df_points = pd.DataFrame(data_dict, index=rain_series["time"].values)
    df_points.index.name = "time"

    results_folder = Path("results")
    results_folder.mkdir(parents=True, exist_ok=True)

    # Folder met de tijdstap als naam
    time_folder = results_folder / f"ens_{ensemble_number}_{safe_time_str}"
    time_folder.mkdir(parents=True, exist_ok=True)
    excel_filename = f"observations_HSAF_points_timeseries_{safe_time_str}.xlsx"
    excel_path = time_folder / excel_filename
    df_points.to_excel(excel_path, index=True)

In [329]:
## Créer Excel avec du nowcast de séries chronologiques
if make_timeseries_Excel:
    nowcast_file = nowcasting_folder / "pysteps_h60b_latest.nc"
    ds_nowcast = xr.open_dataset(nowcast_file)

    varname = "precip_intensity" if "precip_intensity" in ds_nowcast.data_vars else "precipitation"
    var = ds_nowcast[varname]
    ds_nowcast.close()
    if "ens_number" in var.dims:
        var = var.isel(ens_number=ensemble_number)

    data_dict = {}
    y_vals = var["y"].values
    for pt in points_dinteret:
        x_idx = abs(var["x"].values - pt["lon"]).argmin()
        y_idx = abs(var["y"].values - pt["lat"]).argmin()
        y_idx_flipped = len(y_vals) - 1 - y_idx 
        rain_series = var[:, y_idx_flipped, x_idx]
        data_dict[pt["nom"]] = rain_series.values

    first_time = var.time.values[0]
    first_time_str = np.datetime_as_string(first_time, unit='m')
    safe_time_str = first_time_str.replace(":", "").replace("-", "")

    df_points = pd.DataFrame(data_dict, index=rain_series["time"].values)
    df_points.index.name = "time"
    excel_filename = f"nowcast_points_timeseries_{safe_time_str}.xlsx"
    excel_path = time_folder / excel_filename
    df_points.to_excel(excel_path, index=True)

In [330]:
## Créer un graphique avec des données de observation
if make_graph:
    ds_obs = xr.open_mfdataset([str(f) for f in observations_input_nowcast])
    varname = "precip_intensity" if "precip_intensity" in ds_obs.data_vars else "precipitation"
    var = ds_obs[varname]

    data_dict = {}
    for pt in points_dinteret:
        x_idx = abs(var["x"].values - pt["lon"]).argmin()
        y_idx = abs(var["y"].values - pt["lat"]).argmin()
        rain_series = var[:, y_idx, x_idx]
        data_dict[pt["nom"]] = rain_series.values

    df_points = pd.DataFrame(data_dict, index=var.time.values)
    df_points.index.name = "time"

    last_time_dt = pd.to_datetime(var.time.values[-1])
    safe_time_str = last_time_dt.strftime("%Y%m%dT%H%M")
    results_folder = Path("results")
    time_folder = results_folder / f"ens_{ensemble_number}_{safe_time_str}"
    time_folder.mkdir(parents=True, exist_ok=True)

    fig, ax1 = plt.subplots(figsize=(14, 7))
    x = np.arange(len(df_points.index))
    n_locations = len(df_points.columns)
    width = 0.8 / n_locations

    for i, col in enumerate(df_points.columns):
        ax1.bar(
            x + i * width,
            df_points[col].values,
            width=width,
            label=col
        )

    ax1.set_xticks(x + width * (n_locations - 1) / 2)
    ax1.set_xticklabels(pd.to_datetime(df_points.index).strftime("%Y-%m-%d %H:%M"), rotation=45, ha="right")
    ax1.set_ylabel("Intensité des précipitations (mm/h)")
    ax1.set_xlabel("Temps")

    df_mm = df_points * 0.25
    ax2 = ax1.twinx()
    for col in df_mm.columns:
        ax2.plot(
            x,
            df_mm[col].cumsum(),
            linestyle="--",
            label=f"Cumul {col} (mm)"
        )
    ax2.set_ylabel("Somme cumulative (mm)")
    lines_labels = [ax.get_legend_handles_labels() for ax in [ax1, ax2]]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    ax1.legend(lines, labels, loc="upper left")

    plt.title(f"Précipitations et cumul pour toutes les heures")
    plt.tight_layout()

    graph_filename = f"observations_graph_{safe_time_str}.png"
    graph_path = time_folder / graph_filename
    plt.savefig(graph_path, dpi=150)
    plt.close()


In [331]:
## Créer un graphique avec des données de nowcast
if make_graph:
    nowcast_file = nowcasting_folder / "pysteps_h60b_latest.nc"
    ds_nowcast = xr.open_dataset(nowcast_file)

    varname = "precip_intensity" if "precip_intensity" in ds_nowcast.data_vars else "precipitation"
    var = ds_nowcast[varname]
    ds_nowcast.close()
    if "ens_number" in var.dims:
        var = var.isel(ens_number=ensemble_number)
    ds_nowcast.close()

    first_time = var.time.values[0]
    first_time_str = np.datetime_as_string(first_time, unit='m')
    safe_time_str = first_time_str.replace(":", "").replace("-", "")

    data_dict = {}
    y_vals = var["y"].values
    for pt in points_dinteret:
        x_idx = abs(var["x"].values - pt["lon"]).argmin()
        y_idx = abs(var["y"].values - pt["lat"]).argmin()
        y_idx_flipped = len(y_vals) - 1 - y_idx
        rain_series = var[:, y_idx_flipped, x_idx]
        data_dict[pt["nom"]] = rain_series.values

    df_points = pd.DataFrame(data_dict, index=rain_series["time"].values)
    df_points.index.name = "time"
    fig, ax1 = plt.subplots(figsize=(12, 6))
    x = np.arange(len(df_points.index))
    width = 0.8 / len(df_points.columns)

    for i, col in enumerate(df_points.columns):
        ax1.bar(
            x + i * width,
            df_points[col].values,
            width=width,
            label=col
        )

    ax1.set_xticks(x + width * (len(df_points.columns)-1)/2)
    ax1.set_xticklabels(df_points.index.strftime("%Y-%m-%d %H:%M"), rotation=45, ha="right")
    ax1.set_ylabel("Intensité des précipitations (mm/h)")
    ax1.set_xlabel("Temps")
    df_points_mm = df_points * 0.25

    ax2 = ax1.twinx()
    for col in df_points.columns:
        ax2.plot(
            x,
            df_points_mm[col].cumsum(),
            linestyle="--",
            label=f"Cumul {col} (mm)"
        )
    ax2.set_ylabel("Somme cumulative (mm)")

    lines_labels = [ax.get_legend_handles_labels() for ax in [ax1, ax2]]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    ax1.legend(lines, labels, loc="upper left")
    plt.title(f"Intensité de précipitation prévue dans le Nowcast")
    plt.tight_layout()

    graph_filename = f"nowcast_graph_{safe_time_str}.png"
    graph_path = time_folder / graph_filename
    plt.savefig(graph_path, dpi=150)
    plt.close()

In [332]:
if make_gif:
    safe_time_str = target_datetime.strftime("%Y%m%dT%H%M")
    time_folder = results_folder / f"ens_{ensemble_number}_{safe_time_str}"
    gif_files = list(time_folder.glob("*.gif"))

    obs_gif = next((f for f in gif_files if "observations" in f.name.lower()), None)
    nowcast_gif = next((f for f in gif_files if "nowcast" in f.name.lower()), None)

    def add_title(frame, title):
        """Voeg een grote zwarte titel toe bovenaan het frame."""
        frame = frame.convert("RGBA")
        draw = ImageDraw.Draw(frame)
        try:
            font = ImageFont.truetype("arial.ttf", 500)
        except:
            font = ImageFont.load_default()

        text_bbox = draw.textbbox((0, 0), title, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        x = (frame.width - text_width) // 2
        y = 20

        draw.text((x, y), title, fill="black", font=font)
        return frame

    if obs_gif and nowcast_gif:
        gif1 = Image.open(obs_gif)
        gif2 = Image.open(nowcast_gif)

        frames = []
        for frame in ImageSequence.Iterator(gif1):
            frames.append(add_title(frame.copy(), "OBSERVATIONS"))
        for frame in ImageSequence.Iterator(gif2):
            frames.append(add_title(frame.copy(), "NOWCAST"))

        combination_path = time_folder / "combination.gif"
        frames[0].save(
            combination_path,
            save_all=True,
            append_images=frames[1:],
            duration=600,
            loop=0
        )